# Instrumental Variables

In [5]:
import numpy as np
import math 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import scipy.stats as stats
from sklearn import datasets, linear_model

def IV(covXZ=0.5):
    N=500
    alpha = 0.5
    beta = 2.0
    mean = [0, 0, 0]
    cov = [[1, covXZ, 0.7], [covXZ, 1, 0], [0.7, 0, 1]]  # X, Z, eps
    #np.random.seed(123)
    X, Z, eps = np.random.multivariate_normal(mean, cov, N).T

    '''
    plt.scatter(X, Z)
    plt.xlabel("X")
    plt.ylabel("Z")    
    plt.show()
    plt.scatter(X, eps)
    plt.xlabel("X")
    plt.ylabel("epsilon")    
    plt.show()
    plt.scatter(Z, eps)
    plt.xlabel("Z")
    plt.ylabel("epsilon")
    plt.show()
    '''

    print np.corrcoef(X, Z)[0][1], np.corrcoef(X, eps)[0][1], np.corrcoef(Z, eps)[0][1]
    
    Y = [alpha + beta * x + e for x, e in zip(X, eps)]
    
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(Z)
    
    regr = linear_model.LinearRegression()
    regr.fit(X.reshape(N, 1), Y.reshape(N, 1))
    xx = np.linspace(-3, 3, 300)
    xy = [alpha + x * beta for x in xx]    
    
    plt.scatter(X, Y)
    plt.plot(xx, xy)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()
    XYbeta = regr.coef_[0]
    print ("betahat_ols: " + str(XYbeta))
    
    regr.fit(Z.reshape(N, 1), Y.reshape(N, 1))
    zy = [regr.intercept_ + x * regr.coef_[0] for x in xx]
    plt.scatter(Z, Y)
    plt.plot(xx, zy)
    plt.xlabel("Z")
    plt.ylabel("Y")    
    plt.show()
    ZYbeta = regr.coef_[0]    
    print ("betahat_ols: " + str(ZYbeta))
        
        
    regr.fit(Z.reshape(N, 1), X.reshape(N, 1))
    zx = [regr.intercept_ + x * regr.coef_[0] for x in xx]        
    plt.scatter(Z, X)
    plt.plot(xx, zx)
    plt.xlabel("Z")
    plt.ylabel("X")    
    plt.show()
    ZXbeta = regr.coef_[0]
    print ("betahat_ols: " + str(ZXbeta))
    
    regr.fit(X.reshape(N, 1), Z.reshape(N, 1))
    Xhat = regr.predict(Z.reshape(N, 1))
    regr.fit(Y.reshape(N, 1), Z.reshape(N, 1))
    Yhat = regr.predict(Z.reshape(N, 1))    
    
    print ("ratio of ZYbeta/ZXbeta: " + str(ZYbeta / ZXbeta))
    
    plt.plot(Xhat, Yhat, 'ro', label="Xhat ~ Yhat")
    plt.plot(X, Y, 'bo', label="X ~ Y")
    xx = np.linspace(-3, 3, 100)
    yy = [alpha + beta*x for x in xx]
    plt.plot(xx, yy)
    plt.legend()
    plt.show()
    
interact_manual(IV, covXZ=(-0.9, 0.9))

<function __main__.IV>